In [1]:
import cv2
import pytesseract
import imutils
import numpy as np
from PIL import ImageGrab
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import re
import dateutil.parser as dparse

In [4]:
def parseString(string_input):
    try:
        date = dparse.parse(string_input)
        return date
    except ValueError:
        return None

def findDates(string_input):
    copy = string_input
    list = copy.split()
    date_list = []
    while len(list) > 1:
        string = list[0] + list[1]
        dates = parseString(string)
        if dates != None:
            list.pop(0)
            list.pop(0)
            date_list.append(dates)
        else:
            list.pop(0)
    return date_list

def sort_contours(cnts):
    reverse=False
    i=0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    return(cnts, boundingBoxes)

In [5]:
pytesseract.pytesseract.tesseract_cmd = r"/bin/tesseract"
path_pdf = r"test/BBAEngineerResume.pdf"
image = np.asarray(convert_from_path(path_pdf))[0]
greyscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
ret, thresh = cv2.threshold(greyscale, 0, 255, cv2.THRESH_OTSU)
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,50))
contours, hiearchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts, boundingBoxes = sort_contours(contours)
letters = []
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    rect = cv2.rectangle(image, (x,y), (x+w, y+h), (0, 255, 0), 2)
    cropped = image[y:y+h, x:x+w]
    text = pytesseract.image_to_string(cropped)
    
sentences = []
sen = []
for i in text:
    if i != "\n":
        sen.append(i)
    else:
        # print(sen)
        sentences.append(''.join(sen))
        sen=[]

In [7]:
place = None
for i in range(len(sentences)):
    if (re.search("experience", sentences[i], re.IGNORECASE)) != None:
        place = i + 1
        break
    
dates = []
for j in range(place, len(sentences)):
    sen = sentences[j]
    if (re.search("experience", sen, re.IGNORECASE) != None):
        break
    elif (re.search("project", sen, re.IGNORECASE) != None):
        break
    elif (re.search("education", sen, re.IGNORECASE) != None):
        break
    elif (re.search("extracurricular", sen, re.IGNORECASE) != None):
        break
    else:
        dates = dates + findDates(sen)

years = []
for i in range(len(dates)):
    years.append(dates[i].year)

years = np.asarray(years)
print(years)
years = years[years > 999]

# print((np.amax(years)-np.amin(years)) > 9)

[]
